<a href="https://colab.research.google.com/github/GiovanniBorrelli/Software-Requirements-to-User-Stories/blob/main/ConfrontReqs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install nltk rouge-score

# Importazione delle librerie necessarie
import numpy as np
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ERROR: Could not find a version that satisfies the requirement bleurt (from versions: none)
ERROR: No matching distribution found for bleurt


In [32]:
# Carica le user stories dai file (assicurati che i file siano nel tuo ambiente Colab)
with open('llama_chainofthoughts.txt', 'r') as f:
    llama_stories = [story.strip() for story in f.readlines()]

with open('chatgpt_zeroshot.txt', 'r') as f:
    oracle_stories = [story.strip() for story in f.readlines()]

# Rimozione degli spazi bianchi iniziali e finali da ogni User Story
llama_stories = [story.strip() for story in llama_stories]
oracle_stories = [story.strip() for story in oracle_stories]

In [33]:
# Funzione per calcolare BLEU score con smoothing
def calculate_bleu(oracle_stories, llama_stories):
    bleu_scores = []
    smoothie = SmoothingFunction().method4  # Usa un metodo di smoothing
    for ref, cand in zip(oracle_stories, llama_stories):
        bleu_scores.append(sentence_bleu([ref.split()], cand.split(), smoothing_function=smoothie))
    return sum(bleu_scores) / len(bleu_scores)

# Funzione per calcolare ROUGE score per tutte le User Stories
def calculate_rouge(oracle_stories, llama_stories):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, cand in zip(oracle_stories, llama_stories):
        scores = scorer.score(ref, cand)
        for metric in rouge_scores:
            rouge_scores[metric].append(scores[metric].fmeasure)
    return {metric: sum(rouge_scores[metric]) / len(rouge_scores[metric]) for metric in rouge_scores}

# Funzione per calcolare METEOR score con tokenizzazione
def calculate_meteor(oracle_stories, llama_stories):
    meteor_scores = []
    for ref, cand in zip(oracle_stories, llama_stories):
        meteor_scores.append(meteor_score([ref.split()], cand.split()))
    return sum(meteor_scores) / len(meteor_scores)

# Funzione per calcolare Cosine Similarity
def calculate_cosine_similarity(oracle_stories, llama_stories):
    vectorizer = TfidfVectorizer().fit(oracle_stories + llama_stories)
    ref_vectors = vectorizer.transform(oracle_stories)
    gen_vectors = vectorizer.transform(llama_stories)
    similarity_matrix = cosine_similarity(gen_vectors, ref_vectors)
    return np.mean(np.max(similarity_matrix, axis=1))

# Funzione per calcolare F1 Score
def calculate_f1(oracle_stories, llama_stories):
    from sklearn.metrics import f1_score
    f1_scores = []
    for ref, cand in zip(oracle_stories, llama_stories):
        # Converti i testi in etichette binarie
        ref_tokens = set(ref.split())
        cand_tokens = set(cand.split())
        all_tokens = list(ref_tokens.union(cand_tokens))
        ref_vector = [1 if token in ref_tokens else 0 for token in all_tokens]
        cand_vector = [1 if token in cand_tokens else 0 for token in all_tokens]
        f1_scores.append(f1_score(ref_vector, cand_vector))
    return np.mean(f1_scores)

# Calcolo delle metriche
bleu = calculate_bleu(oracle_stories, llama_stories)
rouge = calculate_rouge(oracle_stories, llama_stories)
meteor = calculate_meteor(oracle_stories, llama_stories)
cosine_sim = calculate_cosine_similarity(oracle_stories, llama_stories)
f1 = calculate_f1(oracle_stories, llama_stories)

# Stampa dei risultati
print(f'BLEU Score medio: {bleu:.4f}')
print(f'ROUGE Scores medi: {rouge}')
print(f'METEOR Score medio: {meteor:.4f}')
print(f'Cosine Similarity: {cosine_sim:.4f}')
print(f'F1 Score medio: {f1:.4f}')

BLEU Score medio: 0.0526
ROUGE Scores medi: {'rouge1': 0.3551544977355014, 'rouge2': 0.13139203099503366, 'rougeL': 0.30121539508608686}
METEOR Score medio: 0.2449
Cosine Similarity: 0.5648
F1 Score medio: 0.3221
